<a href="https://colab.research.google.com/github/naafey-aamer/xray_classifier/blob/main/Pneumonia_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

# Load the pre-trained ResNet-50 model (without the top fully connected layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define data directories for train, validation, and test sets
train_data_dir = '/content/drive/MyDrive/xray_pneu/train'
val_data_dir = '/content/drive/MyDrive/xray_pneu/val'
test_data_dir = '/content/drive/MyDrive/xray_pneu/test'

# Set up data generators for train, validation, and test sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']  # Classes should be specified based on subfolder names
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

Mounted at /content/drive
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# print(train_generator[])

In [ ]:
# Freeze pre-trained layers
# for layer in base_model.layers:
#     layer.trainable = False

# Unfreeze all layers
for layer in base_model.layers:
    layer.trainable = True

# keep the bottom layers (first 170) frozen
for layer in base_model.layers[:170]:
    layer.trainable = False

# new dense layers for binary classification
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# new model with the added layers
model = tf.keras.Model(inputs=base_model.input, outputs=output)
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=0.001, momentum=0.9),
    metrics=['accuracy']
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
def lr_schedule(epoch, lr):
    # Reduce learning rate by 10% every 2 epochs
    if (epoch+1) % 2 == 0:
        lr *= 0.9
    return max(lr, 3e-4)

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=0)
earlystop = EarlyStopping(monitor='val_loss', patience=2, verbose=0)

# Fit the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=3,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[lr_callback, earlystop]
)

Epoch 1/3
163/163 [==============================] - 1119s 6s/step - loss: 0.5440 - accuracy: 0.7393 - val_loss: 1.0528 - val_accuracy: 0.5000
Epoch 2/3
163/163 [==============================] - 1020s 6s/step - loss: 0.4345 - accuracy: 0.7897 - val_loss: 0.7635 - val_accuracy: 0.5625
Epoch 3/3
163/163 [==============================] - 1040s 6s/step - loss: 0.3963 - accuracy: 0.8106 - val_loss: 0.6842 - val_accuracy: 0.6250


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_accuracy}')

20/20 [==============================] - 143s 7s/step - loss: 0.4898 - accuracy: 0.7548
Test accuracy: 0.754807710647583
